In [7]:
#Pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

#Torchvision
import torchvision
from torchvision import datasets, models, transforms, utils
  
#Pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

#Torchvision
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader

#Image Processing
import matplotlib.pyplot as plt
from skimage import io, transform, color
import PIL
from PIL import Image
import augmentations
from augmentations import *

#Others
import sklearn.metrics
from sklearn.metrics import *
import numpy as np
import pandas as pd
import cv2
import time
import os
import copy
from model_summary import *
import pretrainedmodels
import tqdm
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings("ignore")

## Dataloader

class dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mask_dir = self.root_dir.replace('CBIS-DDSM_classification','masks')
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.data_frame.iloc[idx]['name'])
        image = Image.open(img_name)
        
        mask_name = os.path.join(self.mask_dir,self.data_frame.iloc[idx]['name'].replace('.j','_mask.j'))
        mask = io.imread(mask_name)
        mask = np.array([mask,mask,mask]).transpose((1,2,0))
        mask = Image.fromarray(mask)

        label = self.data_frame.iloc[idx]['category']       

        if self.transform:
            image,mask = self.transform(image,mask)
        
        mask_final = mask[0,:,:]
        mask_final[mask_final<0.5] = 0
        mask_final[mask_final>0.5] = 1
        
        return {'image':image, 'category':label, 'mask':mask_final, 'name':self.data_frame.iloc[idx]['name']}
    

def get_dataloader(data_dir, train_csv_path, image_size, img_mean, img_std, batch_size=1):

    data_transforms = {
        'train': Compose([
            Resize(image_size),
            RandomHorizontallyFlip(0.5),
            RandomVerticallyFlip(0.5),
            RandomTranslate((0.1,0.1)),
            #RandomRotate(15),
            ToTensor(),
            Normalize(img_mean,img_std)
        ]),
        'valid': Compose([
            Resize(image_size),
            ToTensor(),
            Normalize(img_mean,img_std)
        ]),
        'test': Compose([
            Resize(image_size),
            ToTensor(),
            Normalize(img_mean,img_std)        
        ])
    }

    image_datasets = {}
    dataloaders = {}
    dataset_sizes = {}

    for x in ['train', 'valid', 'test']:
        if x == 'test':
            bs = 1
            sh = False
        else:
            bs = batch_size
            sh = True
        image_datasets[x] = dataset(train_csv_path.replace('train',x),root_dir=data_dir,transform=data_transforms[x])
        dataloaders[x] = torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,shuffle=sh, num_workers=8)    
        dataset_sizes[x] = len(image_datasets[x])

    device = torch.device("cuda:0")

    return dataloaders,dataset_sizes,image_datasets,device

# a,_,_,_ = get_dataloader('../Data/CBIS-DDSM_classification_orient/','../CSV/gain_train.csv',(320,256),[0,0,0],[1,1,1])

# b = iter(a['train']).next()

# c = b['image'].squeeze().numpy().transpose((1,2,0))
# plt.imshow(c)

# c = b['mask'].squeeze().numpy()
# plt.imshow(c)

# b['name']

## Model

# class StdUpsample(nn.Module):
#     def __init__(self, nin, nout):
#         super().__init__()
#         self.conv = nn.ConvTranspose2d(nin, nout, 2, stride=2)
#         self.bn = nn.BatchNorm2d(nout)
        
#     def forward(self, x): return self.bn(F.relu(self.conv(x)))
    
# class VGG_UNet(nn.Module):
#     def __init__(self,base):
#         super().__init__()
#         self.base = base
#         self.conv1 = nn.ConvTranspose2d(512, 256, 2, stride=2)
#         self.conv2 = nn.ConvTranspose2d(256, 256, 2, stride=2)
#         self.conv3 = nn.ConvTranspose2d(256, 1, 2, stride=2)
#         self.bn = nn.BatchNorm2d(256)
        
#     def forward(self,x):
#         x = self.base(x)
#         x = self.bn(F.relu(self.conv1(x)))
#         x = self.bn(F.relu(self.conv2(x)))
#         x = self.bn(F.relu(self.conv2(x)))
#         x = self.bn(F.relu(self.conv2(x)))
#         x = self.conv3(x)
#         return x

# v = models.vgg16_bn(pretrained=True)
# v1 = nn.Sequential(*list(v.children())[:-1])
# model = VGG_UNet(v1)

class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))

class Unet34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[0][i]) for i in [12,22,32,42]]
        self.up1 = UnetBlock(512,512,256)
        self.up2 = UnetBlock(256,512,256)
        self.up3 = UnetBlock(256,256,256)
        self.up4 = UnetBlock(256,128,256)
        self.up5 = nn.ConvTranspose2d(256, 1, 2, stride=2)
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x)
        return x
    
    def close(self):
        for sf in self.sfs: sf.remove()

def build_model():
    v = models.vgg16_bn(pretrained=True)
    v1 = nn.Sequential(*list(v.children())[:-1])
    m = Unet34(v1)
    return m

def get_IoU(pred, targs):
    return 2*(pred*targs).sum() / ((pred+targs).sum())# - (pred*targs).sum())

class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth = 1
        num = targets.size(0)
        probs = F.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score
    
class VGG_unet_bc():
    def __init__(self):
        
        #Initialization
        self.data_dir = '../Data/CBIS-DDSM_classification_orient/'
        self.train_csv = '../CSV/gain_train.csv'
        self.num_epochs = 10
        self.input_shape = (320,256)#(640,512) #(640,512)#(224,224)#(640,384) (640,512)
        self.batch_size = 8
        self.img_mean = [0.485, 0.456, 0.406]#[0,0,0]
        self.img_std = [0.229, 0.224, 0.225]#[1,1,1]
        self.exp_name = 'Weights/VGG_unet_cbis_ddsm'
        
        #Define the three models
        self.model = build_model()
        
        #Put them on the GPU
        self.model = self.model.cuda()
        #self.model.load_state_dict(torch.load('../CSV/models/oxford_fastai.h5'))
        
        #Get the dataloaders
        self.dataloaders,self.dataset_sizes,self.dataset,self.device = get_dataloader(self.data_dir,self.train_csv,\
                                                        self.input_shape,self.img_mean,self.img_std,self.batch_size)
        

        self.optimizer = optim.Adam(self.model.parameters(),lr=1e-2, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
        
        self.lr_scheduler = CosineAnnealingLR(self.optimizer,T_max=10,eta_min=1e-4)
        
        self.loss = nn.BCEWithLogitsLoss()
        
    def train(self):
        
        since = time.time()
        best_epoch_acc = 0.0
        best_epoch_iou = 0.0
        
        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch, self.num_epochs - 1),flush=True)
            print('-' * 10,flush=True)

            # Each epoch has a training and validation phase
            for phase in ['train', 'valid']:
                if phase == 'train':
                    
                    #Set the models to training mode
                    self.lr_scheduler.step()
                    self.model.train()
                
                else:
                    #Set the models to evaluation mode
                    self.model.eval()
                    
                #Keep a track of all the three loss
                running_loss = 0.0
                
                #Metrics : predictor auc and selector iou
                running_iou = 0
                
                #tqdm bar
                pbar = tqdm(total=self.dataset_sizes[phase])

                # Iterate over data.
                for sampled_batch in self.dataloaders[phase]:

                    inputs = sampled_batch['image']
                    labels = sampled_batch['category']
                    mask = sampled_batch['mask']
                    #import pdb;pdb.set_trace()        
                    
                    #Input needs to be float and labels long
                    inputs = inputs.float().to(self.device)
                    labels = labels.long().to(self.device)
                    mask = mask.to(self.device)
                    
                    # zero the parameter gradients
                    self.optimizer.zero_grad()
                
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        
                        outputs = self.model(inputs)
                        
                        out4loss = outputs.view(-1)
                        mask4loss = mask.view(-1)
                        
                        loss = self.loss(out4loss,mask4loss)
                        
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            
                            loss.backward()
                            self.optimizer.step()
                                    
                    preds = out4loss
                    preds[preds>0] = 1
                    preds[preds<=0] = 0

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_iou += get_IoU(out4loss,mask4loss) * inputs.size(0)
                    #print(running_iou)
                    
                    pbar.update(inputs.shape[0])
                pbar.close()

                epoch_loss = running_loss / self.dataset_sizes[phase]
                epoch_iou = running_iou / self.dataset_sizes[phase]
                
                print('{} Sel_Loss: {:.4f} IoU: {:.4f}'.format(
                    phase, epoch_loss, epoch_iou))

                # deep copy the model
                if phase == 'valid' and epoch_iou > best_epoch_iou:
                    best_epoch_iou = epoch_iou
                    torch.save(self.model.state_dict(),self.exp_name+'.pt')
                    
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best IoU: {:4f}'.format(best_epoch_iou))

        torch.save(self.model.state_dict(),self.exp_name+'_final.pt')
        
        print('Training completed finally !!!!!')
        
        
    def test_model(self):
                
        self.model.load_state_dict(torch.load(self.exp_name+'.pt'))
        self.model.eval()
        
        mIoU = 0
        total = 0
        mode = 'test'

        image_list = []
        mask_list = []
        pred_list = []
        
        with torch.no_grad():
            for data in self.dataloaders[mode]:

                images = data['image'].to(self.device)
                mask = denorm_img(data['mask'],self.img_mean,self.img_std)
                
                preds = self.model(images)
                
                image_list.append(images.squeeze().cpu().numpy().transpose((1,2,0)))
                mask_list.append(mask.transpose((1,2,0)).squeeze())
                pred_list.append(preds.squeeze().cpu().numpy())
                
        return image_list,mask_list,pred_list
    

In [8]:
u = VGG_unet_bc()

In [9]:
u.train()

Epoch 0/9
----------


train Sel_Loss: 0.0713 IoU: 0.0034


valid Sel_Loss: 0.0508 IoU: 0.0005
Epoch 1/9
----------


train Sel_Loss: 0.0493 IoU: 0.0007


valid Sel_Loss: 0.0486 IoU: 0.0000
Epoch 2/9
----------


...

valid Sel_Loss: 0.0447 IoU: 0.0952
Epoch 8/9
----------


train Sel_Loss: 0.0439 IoU: 0.0442


valid Sel_Loss: 0.0437 IoU: 0.0557
Epoch 9/9
----------


train Sel_Loss: 0.0437 IoU: 0.0487


valid Sel_Loss: 0.0436 IoU: 0.0550
Training complete in 28m 30s
Best IoU: 0.161987
Training completed finally !!!!!


In [ ]:
'../Data/masks_orient/Mass-Training_P_00913'

In [ ]:
%debug

In [ ]:
a,b,c = u.test_model()

In [ ]:
len(a)

In [ ]:
plt.imshow(a[1])

In [ ]:
b[0].sum()

In [ ]:
plt.imshow(b[1])

In [ ]:
d = c[1]
d[d>0] = 255
d[d<=0] = 0

In [ ]:
plt.imshow(d)

In [ ]:
get_IoU(d/255,b[1])

In [ ]:
plt.imshow(a[1])

In [ ]:
plt.imshow(b[1])

In [ ]:
plt.imshow(c[1])

In [ ]:
u.train()

In [ ]:
%debug

In [ ]:
_,_,_ = dci.get_cam()

In [ ]:
dci.train()

In [ ]:
!nvidia-smi

In [ ]:
# dci.test_model_acc()

In [ ]:
md,dl = dci.return_model()

In [ ]:
a = iter(dl).next()

m = denorm_img(a['mask'],[0.223, 0.231, 0.243],[0.266, 0.270, 0.274]).squeeze()
bm = denorm_img(a['bmask'],[0.223, 0.231, 0.243],[0.266, 0.270, 0.274]).squeeze()

In [ ]:
device = torch.device("cuda:0")

p = F.sigmoid(md(a['image'].to(device))).detach().cpu().numpy().squeeze()

In [ ]:
m.shape

In [ ]:
p.shape

In [ ]:
bm.shape

In [ ]:
plt.imshow(bm)

In [ ]:
plt.imshow(m)

In [ ]:
p_m = p*bm
p_m[p_m > p_m.mean() + p_m.std()] = 1
p_m[p_m < p_m.mean() + p_m.std()] = 0

In [ ]:
plt.imshow(p_m)

In [ ]:
p_m_t = torch.Tensor(p_m)
print(nn.L1Loss()(torch.Tensor(p_m),torch.zeros(p_m_t.shape)))

In [ ]:
p*bm

In [ ]:
plt.imshow(p)